In [3]:


import os
import pandas as pd
import re
from lxml import etree
from datetime import datetime
import logging

file = "../data/output/extractAllPersons.log"

ns = {'tei': 'http://www.tei-c.org/ns/1.0', "xml": "http://www.w3.org/XML/1998/namespace"}
class bcolors:
    HEADER = '\033[95m'
    OKBLUE = '\033[94m'
    OKGREEN = '\033[92m'
    WARNING = '\033[93m'
    FAIL = '\033[91m'
    ENDC = '\033[0m'
    BOLD = '\033[1m'
    UNDERLINE = '\033[4m'

def readlist(pathtofile, filename):
      tree = etree.parse(pathtofile+"/"+filename)
      root = tree.getroot()
      return root

def readfiles(pathtofiles, personlistxml):
    rolenormlist = getrolenormlist()
    listofpersons = []
    for filename in os.listdir(pathtofiles):
       print("--------------------------")
       print(filename)
       regesttree = etree.parse(pathtofiles+"/"+filename)
       regestroot = regesttree.getroot()
       actualregest = regestroot.find(".//tei:idno", ns).text
       date = createdate(regestroot.find(".//tei:creation/tei:date", ns))
       personNodes = regestroot.findall(".//*[@type='person']")
       for rs in personNodes:
           print("---------")
           print("-Node: %s" %rs.tag)
           print("--Attribute: %s" %rs.attrib)
           person = createnewperson(rs, personlistxml, rolenormlist)
           person.append(actualregest)
           person.append(date)
           person.append(getroledead(rs))
           listofpersons.append(person)

    refs = []
    for person in listofpersons:
        if person[0] not in refs:
            refs.append(person[0])
    for ref in refs:
       death = computedeath(ref, listofpersons)
       for person in listofpersons:
           if ref in person:
               person.append(death)

    createcoloumns(listofpersons)

def computedeath(ref, personlist):
        dates =[]
        for person in personlist:
            if ref in person:
                if person[8] == True:
                    if len(person[7]) == 1:
                        date = person[7]
                        dates.append(person[7])
                    elif len(person[7]) == 2:
                        fromtodate = person[7]
                        dates.append(fromtodate[0])
        dates = sorted(dates)
        if len(dates)>0:
            if type(dates) == list:
                date = dates[0]
                return date
            else:
                return dates[0]
        else:
            return " "
def createcoloumns(listofpersons):
    personsFrame = pd.DataFrame()
    reflist = []
    regesttext = []
    sex = []
    regname = []
    title =[]
    title_norm =[]
    title_ref = []
    title_ref_norm = []
    title_off = []
    title_off_norm = []
    title_staff =[]
    title_staff_norm = []
    prof = []
    prof_norm =[]
    eventref = []
    regest = []
    date =[]
    fnrole = []
    death =[]
    dead = []
    fromdate=[]
    todate=[]
    year = []
    toyear =[]
    fromyear =[]


    for person in listofpersons:
        if person[0] != "": reflist.append(person[0])
        else: reflist.append(" ")

        if person[1] != "": regesttext.append(person[1])
        else: reflist.append(" ")

        if len(person[2]) >0:
            if person[2][0] != "": sex.append(person[2][0])
            else: sex.append(" ")
            if person[2][1] != "": regname.append(person[2][1])
            else: regname.append(" ")
        else:
            sex.append(" ")
            regname.append(" ")

        titledict = person[3]
        if len(titledict['title']) >0:
            titlestring = "; ".join(titledict['title'])
            title.append(titlestring)
        else:
            title.append(" ")
        if len(titledict['prof']) >0:
            profstring = "; ".join(titledict['prof'])
            prof.append(profstring)
        else:
            prof.append(" ")
        if len(titledict['ref']) >0:
            title_refstring = "; ".join(titledict['ref'])
            title_ref.append(title_refstring)
        else:
            title_ref.append(" ")
        if len(titledict['off']) >0:
            title_offstring = "; ".join(titledict['off'])
            title_off.append(title_offstring)
        else:
            title_off.append(" ")
        if len(titledict['staff']) >0:
            title_staffstring = "; ".join(titledict['staff'])
            title_staff.append(title_staffstring)
        else:
            title_staff.append(" ")

        if len(titledict['title_norm']) >0:
            title_norm_string = "; ".join(titledict['title_norm'])
            title_norm.append(title_norm_string)
        else:
            title_norm.append(" ")
        if len(titledict['prof_norm']) >0:
            prof_norm_string = "; ".join(titledict['prof_norm'])
            prof_norm.append(prof_norm_string)
        else:
            prof_norm.append(" ")
        if len(titledict['ref_norm']) >0:
            ref_norm_string = "; ".join(titledict['ref_norm'])
            title_ref_norm.append(ref_norm_string)
        else:
            title_ref_norm.append(" ")
        if len(titledict['off_norm']) >0:
            off_norm_string = "; ".join(titledict['off_norm'])
            title_off_norm.append(off_norm_string)
        else:
            title_off_norm.append(" ")
        if len(titledict['staff_norm']) >0:
            staff_norm_string = "; ".join(titledict['staff_norm'])
            title_staff_norm.append(staff_norm_string)
        else:
            title_staff_norm.append(" ")



        if len(person[4]) >0: eventref.append(person[4])
        else: eventref.append(" ")

        if len(person[5]) >0: fnrole.append(person[5])
        else: fnrole.append("no role given")

        if len(person[6]) >0: regest.append(person[6])
        else: regest.append(" ")


        if len(person[7]) == 1:
            persdate = person[7]
            date.append(persdate[0])
            fromdate.append("")
            todate.append("")
            year.append(persdate[0].year)
            toyear.append("")
            fromyear.append("")
        elif len(person[7]) == 2:
            fromtodate = person[7]
            date.append("")
            fromdate.append(fromtodate[0])
            todate.append(fromtodate[1])
            year.append("")
            toyear.append(fromtodate[0].year)
            fromyear.append(fromtodate[1].year)

        dead.append(person[8])

        death.append(person[9])

    personsFrame['ref'] = reflist
    personsFrame['Regest Text'] = regesttext
    personsFrame['reg Name'] = regname
    personsFrame['Sex'] = sex
    personsFrame['Titles'] = title
    personsFrame['Titles norm'] = title_norm
    personsFrame['Titles ref'] = title_ref
    personsFrame['Titles ref norm'] = title_ref_norm
    personsFrame['Professions'] = prof
    personsFrame['Professions norm'] = prof_norm
    personsFrame['staff'] = title_staff
    personsFrame['staff norm'] = title_staff_norm
    personsFrame['off'] = title_off
    personsFrame['off norm'] = title_off_norm
    personsFrame['fn role'] = fnrole
    personsFrame['Events ref']  = eventref
    personsFrame['regest'] = regest
    personsFrame['regest date'] = date
    personsFrame['from'] = fromdate
    personsFrame['to'] = todate
    personsFrame['dead'] = dead
    personsFrame['earliest possible death'] = death
    personsFrame['year'] = year
    personsFrame['fromyear'] = fromyear
    personsFrame['toyear'] = toyear

    writer = pd.ExcelWriter('../data/output/xlsx/extractedAllPersons.xlsx')
    personsFrame.to_excel(writer, 'ExtractedAllPersons', index=False)
    writer.save()

    personsFrame.to_csv("../data/output/csv/extractedAllPersons.csv", sep="\t", encoding="utf-8")


def createnewperson(rs, personlistxml, rolenormlist):
        text = cleanuptextnodestostring(rs.xpath(".//text()"))
        print("--Text: %s" %text)
        roles = getpersonroles(rs, rolenormlist)
        rsrole = getrstyperole(rs)
        events = getpersonevent(rs)
        if rs.attrib["ref"] is not None:
            ref = rs.attrib["ref"].replace("#","")
        else:
            print(bcolors.FAIL+"<rs type='person'> ref nicht gefunden")

        person = getinformationsfrompersonlistxml(ref, personlistxml)
        return [ref, text, person, roles, events, rsrole]

def getrstyperole(persnode):
    parentrs = persnode.xpath("ancestor::*[@type='fn']")
    print("--@fn-Parents")
    for parent in parentrs:
        print("---Node: %s" %parent.tag)
        print("----Attribute: %s" %parent.attrib)
    if len(parentrs) >0:
        return parentrs[0].attrib['role']
    else:
        return "None"

def getroledead(persnode):
    deadnode  = persnode.find("./tei:roleName[@type='dead']",ns)
    if deadnode is not None:
        return True
    else:
        return False

def getpersonroles(persnode, rolenormlist):
    roles = {}
    rolenamenodes = persnode.findall(".//tei:roleName", ns)
    print("--roleNames:")
    if rolenamenodes is not None:
        roles_prof = []
        roles_title = []
        roles_ref = []
        roles_off = []
        roles_staff= []
        roles_prof_norm = []
        roles_title_norm = []
        roles_ref_norm = []
        roles_off_norm = []
        roles_staff_norm= []
        for rolenamenode in rolenamenodes:
            print("---Node: %s"%rolenamenode.tag)
            print("----Attribute: %s"%rolenamenode.attrib)
            content = cleanuptextnodestostring(rolenamenode.xpath(".//text()"))
            print("----Content: %s" %content)
            if rolenamenode.attrib['type'] == "prof":
                rolelist = normrole(rolenormlist, content)
                roles_prof.append(rolelist[0])
                roles_prof_norm.append(rolelist[1])
            elif rolenamenode.attrib['type'] == "title":
                rolelist = normrole(rolenormlist, content)
                roles_title.append(rolelist[0])
                roles_title_norm.append(rolelist[1])
            elif rolenamenode.attrib['type'] == "ref":
                rolelist = normrole(rolenormlist, content)
                roles_ref.append(rolelist[0])
                roles_ref_norm.append(rolelist[1])
            elif rolenamenode.attrib['type'] == "off":
                rolelist = normrole(rolenormlist, content)
                roles_off.append(rolelist[0])
                roles_off_norm.append(rolelist[1])
            elif rolenamenode.attrib['type'] == "staff":
                rolelist = normrole(rolenormlist, content)
                roles_staff.append(rolelist[0])
                roles_staff_norm.append(rolelist[1])

    roles['title'] = roles_title
    roles['ref'] = roles_ref
    roles['prof'] = roles_prof
    roles['off'] = roles_off
    roles['staff'] = roles_staff
    roles['title_norm'] = roles_title_norm
    roles['ref_norm'] = roles_ref_norm
    roles['prof_norm'] = roles_prof_norm
    roles['off_norm'] = roles_off_norm
    roles['staff_norm'] = roles_staff_norm
    return roles

def getpersonevent(persnode):
    eventnode = getparentevent(persnode)
    if eventnode is not None:
        if 'ref' in eventnode.attrib:
            event_ref = eventnode.attrib['ref']
            return event_ref
        else: return ""
    else: return ""


def getparentevent(node):
        actualnode = node
        parents = actualnode.xpath("ancestor::*[@type='event']")
        if len(parents) >0:
            return parents[0]
        else: return None




def createdate(node):
    print("Datum: %s" %node.attrib)
    if 'when' in node.attrib:
        when = node.attrib['when']
        year = node.attrib['when'][:4]
        date = datetime.strptime(when, '%Y%m%d').date()
        return [date]
    elif 'from' in node.attrib:
        fr =node.attrib['from']
        to = node.attrib['to']
        fromdate = datetime.strptime(fr, '%Y%m%d').date()
        todate = datetime.strptime(to, '%Y%m%d').date()

        if fromdate == todate:
            return[fromdate]
        else: return [fromdate, todate]



def getinformationsfrompersonlistxml(ref, personlist):
    ref = ref.replace("#", '')
    person = personlist.find(".//tei:person[@xml:id='"+ref+"']", ns)
    if person is not None:
        if "sex" in person.attrib:
            sex = person.attrib['sex']
        else: sex = "-"
        persname = person.find(".//tei:persName", ns)
        text = cleanuptextnodestostring(persname.xpath(".//text()"))
        return [sex, text]
    else:
        return []



def cleanuptextnodestostring(textnodes):
    cleanedlist =[]
    for textnode in textnodes:
        textstring = str(textnode)
        tokenized = textstring.split(" ")
        cleandtokens = []
        for token in tokenized:
            if len(token) >1:
                token = re.sub("\n","", token)
                cleandtokens.append(token)
        cleanedlist += cleandtokens
    return " ".join(cleanedlist)

def getrolenormlist():
    role_mapping = pd.read_csv("https://docs.google.com/spreadsheets/d/1_ygvqRnDTg6Rx2wp-RNPvZ3cKQFxfDPo3d7SBDiLO5c/export?gid=0&format=csv", dtype=str)\
    .iloc[:,1:3]
    rolesnormlist = role_mapping.set_index("role").to_dict()

    return rolesnormlist['role_norm']


def normrole(rolenormlist, role):
    if role.lower() in rolenormlist:
        return [role, rolenormlist[role.lower()]]
    elif role in rolenormlist:
        return [role, rolenormlist[role]]
    else:
        return [role, " "]

personlistxml = readlist("../../data/lists", "personList.xml")
readfiles("../../regests_QGW_PR_Stadt_ab_1400/done", personlistxml)


--------------------------
1482.xml
Datum: {'when': '14010119'}
---------
-Node: {http://www.tei-c.org/ns/1.0}rs
--Attribute: {'type': 'person', 'ref': '#pe__wilhelm_herzog_von_oesterreich'}
--Text: Herzog Wilhalm
--roleNames:
--@fn-Parents
---Node: {http://www.tei-c.org/ns/1.0}rs
----Attribute: {'type': 'fn', 'role': 'issuer'}
---------
-Node: {http://www.tei-c.org/ns/1.0}rs
--Attribute: {'type': 'person', 'ref': '#pe__heinrich_egelhaimer_QGW_II_I_1482'}
--Text: saumerkneeht Hainreichen dem Egelhaimer
--roleNames:
---Node: {http://www.tei-c.org/ns/1.0}roleName
----Attribute: {'type': 'staff', 'corresp': '#pe__wilhelm_herzog_von_oesterreich'}
----Content: saumerkneeht
--@fn-Parents
---Node: {http://www.tei-c.org/ns/1.0}rs
----Attribute: {'type': 'fn', 'role': 'recipient'}
--------------------------
1483.xml
Datum: {'when': '14010128'}
---------
-Node: {http://www.tei-c.org/ns/1.0}rs
--Attribute: {'type': 'person', 'ref': '#pe__haunold_schuchler_II_QGW_II_I_1557'}
--Text: Haunolt der Sc

--@fn-Parents
---Node: {http://www.tei-c.org/ns/1.0}rs
----Attribute: {'type': 'fn', 'role': 'witness'}
---------
-Node: {http://www.tei-c.org/ns/1.0}rs
--Attribute: {'type': 'person', 'ref': '#pe__johann_wlassivis'}
--Text: Johannes, Johannis de Wlassivis, clericus Pragensis diocesis publicus apostolica et imperiali auctoritate notarius
--roleNames:
---Node: {http://www.tei-c.org/ns/1.0}roleName
----Attribute: {'type': 'off', 'corresp': '#org__nalb_pfarre'}
----Content: clericus
---Node: {http://www.tei-c.org/ns/1.0}roleName
----Attribute: {'type': 'off', 'corresp': '#org__vatikan'}
----Content: publicus apostolica
---Node: {http://www.tei-c.org/ns/1.0}roleName
----Attribute: {'type': 'off', 'corresp': '#org__heilige_roemische_reich'}
----Content: imperiali auctoritate notarius
--@fn-Parents
---Node: {http://www.tei-c.org/ns/1.0}rs
----Attribute: {'type': 'fn', 'role': 'witness'}
--------------------------
1488.xml
Datum: {'when': '14010428'}
---------
-Node: {http://www.tei-c.org/ns/

--------------------------
1491.xml
Datum: {'when': '14010804'}
---------
-Node: {http://www.tei-c.org/ns/1.0}rs
--Attribute: {'type': 'person', 'ref': '#pe__michael_affer_QGW_II_I_1491'}
--Text: Michel der Affer, purger ze Wienn
--roleNames:
---Node: {http://www.tei-c.org/ns/1.0}roleName
----Attribute: {'type': 'titel_ref', 'corresp': '#org__wien'}
----Content: purger
--@fn-Parents
---Node: {http://www.tei-c.org/ns/1.0}rs
----Attribute: {'type': 'fn', 'role': 'issuer'}
---------
-Node: {http://www.tei-c.org/ns/1.0}rs
--Attribute: {'type': 'person', 'ref': '#pe__christine_QGW_II_I_1491'}
--Text: Christein, seine Hausfrau
--roleNames:
---Node: {http://www.tei-c.org/ns/1.0}roleName
----Attribute: {'type': 'kin', 'corresp': '#pe__michael_affer_QGW_II_I_1491'}
----Content: Hausfrau
--@fn-Parents
---Node: {http://www.tei-c.org/ns/1.0}rs
----Attribute: {'type': 'fn', 'role': 'issuer'}
---------
-Node: {http://www.tei-c.org/ns/1.0}rs
--Attribute: {'type': 'person', 'ref': '#pe__berthold_lang_

--------------------------
1495.xml
Datum: {'when': '14010920'}
---------
-Node: {http://www.tei-c.org/ns/1.0}rs
--Attribute: {'type': 'person', 'ref': '#pe__agnes_QGW_II_I_1495'}
--Text: Angnes, Eberharts, des schuster wittib wittib
--roleNames:
---Node: {http://www.tei-c.org/ns/1.0}roleName
----Attribute: {'type': 'prof'}
----Content: schuster
---Node: {http://www.tei-c.org/ns/1.0}roleName
----Attribute: {'type': 'dead'}
----Content: wittib
---Node: {http://www.tei-c.org/ns/1.0}roleName
----Attribute: {'type': 'kin', 'corresp': '#pe__eberhard_der_schuster_QGW_II_I_1495'}
----Content: wittib
--@fn-Parents
---Node: {http://www.tei-c.org/ns/1.0}rs
----Attribute: {'type': 'fn', 'role': 'issuer'}
---------
-Node: {http://www.tei-c.org/ns/1.0}rs
--Attribute: {'type': 'person', 'ref': '#pe__eberhard_der_schuster_QGW_II_I_1495'}
--Text: Eberharts, des schuster wittib
--roleNames:
---Node: {http://www.tei-c.org/ns/1.0}roleName
----Attribute: {'type': 'prof'}
----Content: schuster
---Node: {ht

--------------------------
1498.xml
Datum: {'when': '14011214'}
---------
-Node: {http://www.tei-c.org/ns/1.0}rs
--Attribute: {'type': 'person', 'ref': '#pe__elisabeth_QGW_II_I_1498'}
--Text: Elspet, witib Ulreichs witib, des kerzenmacher
--roleNames:
---Node: {http://www.tei-c.org/ns/1.0}roleName
----Attribute: {'type': 'kin', 'corresp': '#pe__ulrich_QGW_II_I_1498'}
----Content: witib
---Node: {http://www.tei-c.org/ns/1.0}roleName
----Attribute: {'type': 'prof'}
----Content: kerzenmacher
--@fn-Parents
---Node: {http://www.tei-c.org/ns/1.0}rs
----Attribute: {'type': 'fn', 'role': 'issuer'}
---------
-Node: {http://www.tei-c.org/ns/1.0}rs
--Attribute: {'type': 'person', 'ref': '#pe__ulrich_QGW_II_I_1498'}
--Text: Ulreichs witib, des kerzenmacher
--roleNames:
---Node: {http://www.tei-c.org/ns/1.0}roleName
----Attribute: {'type': 'prof'}
----Content: kerzenmacher
--@fn-Parents
---Node: {http://www.tei-c.org/ns/1.0}rs
----Attribute: {'type': 'fn', 'role': 'issuer'}
---------
-Node: {http:/

...

---------
-Node: {http://www.tei-c.org/ns/1.0}rs
--Attribute: {'type': 'person', 'ref': '#pe__konrad_vorlauf'}
--Text: Bürgermeisters hern Chunraten des Vorlaufs
--roleNames:
---Node: {http://www.tei-c.org/ns/1.0}roleName
----Attribute: {'type': 'off', 'corresp': '#org__wien'}
----Content: Bürgermeisters
---Node: {http://www.tei-c.org/ns/1.0}roleName
----Attribute: {'type': 'title'}
----Content: hern
--@fn-Parents
---Node: {http://www.tei-c.org/ns/1.0}rs
----Attribute: {'type': 'fn', 'role': 'other'}
---------
-Node: {http://www.tei-c.org/ns/1.0}rs
--Attribute: {'type': 'person', 'ref': '#pe__friedrich_frosch'}
--Text: hern Fridreichen dem Frosch, Kaplan der Messe, die frau Ann, weilent Niklas des Rorcholben hausfrau, den baiden got gnad, auf sand Marien Magdalen altar, gelegen auf dem neuen Karner dacz sand Stephan, gestiftet hatte.
--roleNames:
---Node: {http://www.tei-c.org/ns/1.0}roleName
----Attribute: {'type': 'title'}
----Content: hern
---Node: {http://www.tei-c.org/ns/1.0}roleN

---------
-Node: {http://www.tei-c.org/ns/1.0}rs
--Attribute: {'type': 'person', 'ref': '#pe__andreas_an_dem_st_petersfreithof'}
--Text: Andre an sand Peters freithof, chelermaister in Osterreich
--roleNames:
---Node: {http://www.tei-c.org/ns/1.0}roleName
----Attribute: {'type': 'off', 'corresp': '#org__oesterreich_herzogtum'}
----Content: chelermaister
--@fn-Parents
---Node: {http://www.tei-c.org/ns/1.0}rs
----Attribute: {'type': 'fn', 'role': 'recipient'}
---------
-Node: {http://www.tei-c.org/ns/1.0}rs
--Attribute: {'type': 'person', 'ref': '#pe__rudolf_angerfelder'}
--Text: Rudolf Angerfelder, des rats der stat ze Wienn
--roleNames:
---Node: {http://www.tei-c.org/ns/1.0}roleName
----Attribute: {'type': 'off', 'corresp': '#org__wien'}
----Content: rats der stat
--@fn-Parents
---Node: {http://www.tei-c.org/ns/1.0}rs
----Attribute: {'type': 'fn', 'role': 'recipient'}
--------------------------
1597.xml
Datum: {'when': '14041025'}
---------
-Node: {http://www.tei-c.org/ns/1.0}rs
--Attr

---------
-Node: {http://www.tei-c.org/ns/1.0}rs
--Attribute: {'type': 'person', 'ref': '#pe__johann_siedendorfer_QGW_II_I_1600'}
--Text: Hannsen dem Siedendorfer, dem kürsner
--roleNames:
---Node: {http://www.tei-c.org/ns/1.0}roleName
----Attribute: {'type': 'prof'}
----Content: kürsner
--@fn-Parents
---Node: {http://www.tei-c.org/ns/1.0}rs
----Attribute: {'type': 'fn', 'role': 'other'}
---------
-Node: {http://www.tei-c.org/ns/1.0}rs
--Attribute: {'type': 'person', 'ref': '#pe__ulrich_werder_QGW_II_I_1600'}
--Text: Kläger
--roleNames:
--@fn-Parents
---Node: {http://www.tei-c.org/ns/1.0}rs
----Attribute: {'type': 'fn', 'role': 'recipient'}
---------
-Node: {http://www.tei-c.org/ns/1.0}rs
--Attribute: {'type': 'person', 'ref': '#pe__peter_angerfelder'}
--Text: Aussteller
--roleNames:
--@fn-Parents
---Node: {http://www.tei-c.org/ns/1.0}rs
----Attribute: {'type': 'fn', 'role': 'witness'}
--------------------------
1601.xml
Datum: {'when': '14041029'}
---------
-Node: {http://www.tei-c.or

---------
-Node: {http://www.tei-c.org/ns/1.0}rs
--Attribute: {'type': 'person', 'ref': '#pe__johann_teschinger_QGW_II_I_1604'}
--Text: Hannsen des Teschinger, vogt
--roleNames:
---Node: {http://www.tei-c.org/ns/1.0}roleName
----Attribute: {'type': 'off', 'corresp': '#org__augsburg'}
----Content: vogt
--@fn-Parents
---Node: {http://www.tei-c.org/ns/1.0}rs
----Attribute: {'type': 'fn', 'role': 'witness'}
---------
-Node: {http://www.tei-c.org/ns/1.0}rs
--Attribute: {'type': 'person', 'ref': '#pe__konrad_oeffenlin_QGW_II_I_1604'}
--Text: Chunrats des Öffenlins, burger zu Augspurg
--roleNames:
---Node: {http://www.tei-c.org/ns/1.0}roleName
----Attribute: {'type': 'title_ref', 'corresp': '#org__augsburg'}
----Content: burger
--@fn-Parents
---Node: {http://www.tei-c.org/ns/1.0}rs
----Attribute: {'type': 'fn', 'role': 'witness'}
--------------------------
1605.xml
Datum: {'when': '14041127'}
---------
-Node: {http://www.tei-c.org/ns/1.0}rs
--Attribute: {'type': 'person', 'ref': '#pe__wilhelm

--------------------------
1609.xml
Datum: {'when': '14041216'}
---------
-Node: {http://www.tei-c.org/ns/1.0}rs
--Attribute: {'type': 'person', 'ref': '#pe__friedrich_perolt_QGW_II_I_1609'}
--Text: Fridreich Perolt, genant Fricz Paternostrer, purger ze Wienn
--roleNames:
---Node: {http://www.tei-c.org/ns/1.0}roleName
----Attribute: {'type': 'title_ref', 'corresp': '#org__wien'}
----Content: purger
--@fn-Parents
---Node: {http://www.tei-c.org/ns/1.0}rs
----Attribute: {'type': 'fn', 'role': 'issuer'}
---------
-Node: {http://www.tei-c.org/ns/1.0}rs
--Attribute: {'type': 'person', 'ref': '#pe__katharina_QGW_II_I_1609'}
--Text: Kathrei, wittib Fridreichs, des tischer wittib den man genant hat Fricz Tischer anstatt anstatt
--roleNames:
---Node: {http://www.tei-c.org/ns/1.0}roleName
----Attribute: {'type': 'kin', 'corresp': '#pe__friedrich_der_tischler_QGW_II_I_1609'}
----Content: wittib
---Node: {http://www.tei-c.org/ns/1.0}roleName
----Attribute: {'type': 'prof'}
----Content: tischer
---N